# PyCity Schools Analysis

* Across the board, schools with lower budget per student out performed schools with higher budgets per student. This leads me to want to do further analysis on how budgets are utilized, but does allow me to wonder whether budget is really a factor at all in test scores.

* As a whole, schools with under 2000 students (small and medium) dramatically out-performed larger sized schools. It would be interesting to know if class sizes were accordingly smaller and what the teacher to student ratios are at the smaller verses larger schools. 

* Across the board, charter schools out-performed the district schools. It is also apparent from the data (top performing schools) that the top schools are also at (or just above ~2200) or under 2000 students. We could be looking at a size factor again with this statistic.

---

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary
* Create a high level snapshot (in table form) of the district's key metrics, including:
  * Total Schools
  * Total Students
  * Total Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results



In [2]:
#calculate necessary values
unique_schools = school_data_complete["School ID"].nunique()
tot_student_count = school_data_complete["Student ID"].nunique()
tot_budget = school_data['budget'].sum()
avg_math_score = school_data_complete['math_score'].mean()
avg_reading_score = school_data_complete['reading_score'].mean()
overall_avg_score = (school_data_complete['math_score'].mean()+school_data_complete['reading_score'].mean())/2
school_data_complete_passing_math = school_data_complete[school_data_complete.math_score >= 70]
pass_math_tot = school_data_complete_passing_math['Student ID'].nunique()
school_data_complete_passing_reading = school_data_complete[school_data_complete.reading_score >= 70]
pass_reading_tot = school_data_complete_passing_reading['Student ID'].nunique()

In [3]:
#create a dataframe for data
dist_summary_table = pd.DataFrame({"Total Schools":unique_schools,
                              "Total Students":[tot_student_count],
                              "Total Budget":[tot_budget],
                              "Average Math Score":[avg_math_score],
                              "Average Reading Score":[avg_reading_score],
                              "% Passing Math":[pass_math_tot/tot_student_count],
                              "% Passing Reading":[pass_reading_tot/tot_student_count],
                              "% Overall Passing":[((pass_math_tot/tot_student_count)+(pass_reading_tot/tot_student_count))/2]})
#dist_summary_table

In [4]:
#format the df
dist_summary_table.style.format({'% Passing Math': '{:.4%}',
                                 '% Passing Reading': '{:.4%}',
                                 '% Overall Passing': '{:.4%}',
                                 'Total Students': '{:,}',
                                 'Total Budget': '${:,}'})


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.9854,81.8778,74.9809%,85.8055%,80.3932%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [5]:
#create a new dataframe grabbing aggregate values first
school_summary_table = school_data_complete.groupby(['school_name', 'type'],as_index=False).agg({'Student ID':'count',
                                                                                                 'budget':'mean',
                                                                                                 'math_score':'mean',
                                                                                                 'reading_score': 'mean'})
#rename columns in df
school_summary_table = school_summary_table.rename(columns = {"Student ID": "Total Students",
                                                              "budget": "Total Budget",
                                                              "school_name": "School Name",
                                                              "type": "School Type",
                                                              "math_score": "Average Math Score",
                                                              "reading_score": "Average Reading Score"})

#calculate budget per student and add to df
z = school_summary_table["Total Budget"]/school_summary_table["Total Students"]
school_summary_table["Per Student Budget"] = z

#grab the count of students who passed math and add a temporary col to the df
groupby_passing_math = school_data_complete[school_data_complete.math_score >= 70].groupby('school_name')
school_summary_table['Total Passing Math'] = groupby_passing_math['Student ID'].count().values

#grab the count of students who passed reading and add a temporary col to the df
groupby_passing_reading = school_data_complete[school_data_complete.reading_score >= 70].groupby('school_name')
school_summary_table['Total Passing Reading'] = groupby_passing_reading['Student ID'].count().values

#calculate pecentages of students passing math and reading and add to the df
x = (school_summary_table['Total Passing Math']/school_summary_table['Total Students'])*100
y = (school_summary_table['Total Passing Reading']/school_summary_table['Total Students'])*100
school_summary_table['% Passing Math'] = x
school_summary_table['% Passing Reading'] = y
school_summary_table['Overall Passing Rate (%)'] = (x+y)/2


In [6]:
#drop the temporary columns and reorganize columns for cleaner appearance
school_summary_table.drop('Total Passing Math', axis=1, inplace=True) 
school_summary_table.drop('Total Passing Reading', axis=1, inplace=True)
school_summary_table=school_summary_table[['School Name', 'School Type', 'Total Students', 'Total Budget',
                                           'Per Student Budget', 'Average Math Score', 'Average Reading Score',
                                           '% Passing Math', '% Passing Reading', 'Overall Passing Rate (%)']]
#set the index to the School Name
school_summary_table.set_index('School Name', inplace=True)

In [7]:
#format the df
school_summary_table.style.format({'Total Students': '{:,}',
                                   'Total Budget': '${:,.2f}',
                                   'Per Student Budget': '${:,.2f}',
                                   'Average Math Score': '{:.6f}',
                                   'Average Reading Score': '{:.6f}',
                                   '% Passing Math': '{:.6f}',
                                   '% Passing Reading': '{:.6f}',
                                   'Overall Passing Rate (%)': '{:.6f}'})

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [8]:
school_summary_table_top = school_summary_table.sort_values("Overall Passing Rate (%)", ascending=False)
school_summary_table_top.head(5).style.format({'Total Students': '{:,}',
                                   'Total Budget': '${:,.2f}',
                                   'Per Student Budget': '${:,.2f}',
                                   'Average Math Score': '{:.6f}',
                                   'Average Reading Score': '{:.6f}',
                                   '% Passing Math': '{:.6f}',
                                   '% Passing Reading': '{:.6f}',
                                   'Overall Passing Rate (%)': '{:.6f}'})

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [9]:
school_summary_table_bottom = school_summary_table.sort_values("Overall Passing Rate (%)", ascending=True)
school_summary_table_bottom.head(5).style.format({'Total Students': '{:,}',
                                   'Total Budget': '${:,.2f}',
                                   'Per Student Budget': '${:,.2f}',
                                   'Average Math Score': '{:.6f}',
                                   'Average Reading Score': '{:.6f}',
                                   '% Passing Math': '{:.6f}',
                                   '% Passing Reading': '{:.6f}',
                                   'Overall Passing Rate (%)': '{:.6f}'})

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [10]:
schools_by_grade = school_data_complete.groupby(["school_name", "grade"],as_index=True)

math_by_grade = schools_by_grade['math_score'].mean().unstack(level=1)
math_by_grade = math_by_grade[['9th','10th', '11th', '12th']]
math_by_grade

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [11]:
reading_by_grade = schools_by_grade['reading_score'].mean().unstack(level=1)
reading_by_grade = math_by_grade[['9th','10th', '11th', '12th']]
reading_by_grade

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [12]:
#define new dataframe w/ necessary columns
scores_by_school_spending = school_summary_table.loc[:,['Per Student Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate (%)']]

# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-614", "$615-644", "$645-675"]
scores_by_school_spending["Spending Ranges(Per Student)"] = pd.cut(scores_by_school_spending["Per Student Budget"], spending_bins, labels=group_names, right=False)
#scores_by_school_spending

In [13]:
group_df = scores_by_school_spending.groupby("Spending Ranges(Per Student)")
df = group_df.aggregate(np.mean)
df.drop('Per Student Budget', axis=1, inplace=True) 
df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
Spending Ranges(Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-614,83.599686,83.885211,94.230858,95.900287,95.065572
$615-644,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [14]:
#need data from two dataframes; grab necessary columns and merge dfs
school_data_new = school_data[["school_name", "size"]]
school_summary_new = school_summary_table[['Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate (%)']]
scores_by_size = pd.merge(school_summary_new, school_data_new, left_on="School Name", right_on="school_name")

# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
scores_by_size["School Size"] = pd.cut(scores_by_size["size"], size_bins, labels=group_names, right=False)
#scores_by_size

In [15]:
group_df = scores_by_size.groupby("School Size")
df = group_df.aggregate(np.mean)
df.drop('size', axis=1, inplace=True) 
df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [16]:
#define new dataframe w/ necessary columns
scores_by_school_type = school_summary_table[['School Type','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate (%)']]

group_df = scores_by_school_type.groupby("School Type")
df = group_df.aggregate(np.mean)
df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
